In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import requests
import folium
import matplotlib.pyplot as plt
import datetime

In [2]:
# 구분 리스트 url : https://www.starbucks.co.kr/store/getGugunList.do
# 구분 리스트 호출 파라메터: sido_cd=08&rndCod=O1QIKXA86D
# sido : 01 ~ 17

# 스토어 검색 url : https://www.starbucks.co.kr/store/getStore.do
# 스토어 호출 파라메터
    # p_sido_cd: 08 / searchType: C

In [3]:
# 기본 데이터 세팅

store_url = 'https://www.starbucks.co.kr/store/getStore.do?r=A49TWAG7RG'
sido = '01'
headers = {"USER-AGENT" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}

In [4]:
# 결과 json 테이블에서 사용할 컬럼 세팅

cols = ['s_code','s_name','tel','sido_name','addr','doro_address','open_dt','lat','lot']

In [5]:
# 시도별 점포 검색 함수

def get_sido_store(sido) :
    search_url = f'{store_url}&in_biz_cds=0&in_scodes=0&ins_lat=37.56682&ins_lng=126.97865&search_text=&p_sido_cd={sido}&p_gugun_cd=&isError=true&in_distance=0&in_biz_cd=&iend=1000&searchType=C&set_date=&rndCod=6ZRZX5XQ84&all_store=0&T03=0&T01=0&T27=0&T12=0&T09=0&T30=0&T05=0&T22=0&T21=0&T10=0&T36=0&T43=0&T48=0&P10=0&P50=0&P20=0&P60=0&P30=0&P70=0&P40=0&P80=0&whcroad_yn=0&P90=0&new_bool=0'
    response = requests.post(search_url, headers = headers)
    return pd.DataFrame(response.json()['list'])[cols]    

In [6]:
# 시도별 점포 검색 후 한 테이블로 합치기
store_temp = []

for i in range(0, 17) :
    sido = str(i+1).zfill(2)
    store_temp.append(get_sido_store(sido))

In [7]:
store_list = pd.concat(store_temp, axis = 0, ignore_index=True)

In [8]:
# 테이블 정리
# 컬럼명 변경
# 컬럼별 데이터 타입 변경

store_list = store_list.rename(columns = {'s_code' : '점포번호',
                                        's_name' : '점포명',
                                        'tel' : '전화번호',
                                        'sido_name' : '지역',
                                        'addr' : '주소(지번)',
                                        'doro_address' : '주소(도로명)',
                                        'open_dt' : '개장일',
                                        'lat':'위도',
                                        'lot': '경도'})

store_list['위도'] = store_list['위도'].astype('float')
store_list['경도'] = store_list['경도'].astype('float')
store_list['점포번호'] = store_list['점포번호'].astype('int')
store_list['개장일'] = store_list['개장일'].map(lambda x : datetime.datetime.strptime(x, '%Y%m%d'))

In [9]:
# 지도 설정을 위한 시작 지점 설정
# 서울에 데이터가 몰려 있기 때문에, 중간값 설정을 위해서 최대 최소값 구한 후 중간 지점 설정

start_lot = (store_list['경도'].max() + store_list['경도'].min()) / 2
start_lat = (store_list['위도'].max() + store_list['위도'].min()) / 2

In [10]:
map = folium.Map(location=[start_lat, start_lot], zoom_start=7)

for j in store_list.index :
    name = store_list.loc[j, '점포명']
    addr = store_list.loc[j, '주소(도로명)']
    pop = f'{name}\n{addr}'
    location = [store_list.loc[j,'위도'], store_list.loc[j,'경도']]
    
    folium.Marker(
                location = location,
                popup = pop
                ).add_to(map)

map